In [3]:
import numpy as np
import matplotlib.pyplot as plt
from microboone_utils import *
import pandas as pd
from h5py import File

In [4]:
#abre o arquivo
f = File('bnb_WithWire_00.h5','r')

In [5]:
#mostra todas as tabelas disponiveis no arquivo .h5 aberto

labels=[]
sub_labels=[]
for lines in f:
        labels.append(lines)
        aux=[]
        for key in f[lines]:
            aux.append(key)
        sub_labels.append(aux)
labels

['edep_table',
 'event_table',
 'hit_table',
 'opflash_table',
 'opflashsumpe_table',
 'ophit_table',
 'pandoraHit_table',
 'pandoraPfp_table',
 'pandoraPrimary_table',
 'particle_table',
 'wire_table']

In [6]:
#montar uma tavela dos eventos disponiveis na file
size=len(f[labels[1]]["event_id"][:])

names_event=["run","subrun","event"]
names_event_values=[[] for _ in range(len(names_event))]
for i in range(size):
    for j in range(len(names_event)):
        names_event_values[j].append(f[labels[1]]["event_id"][i][j])

temp_dict = {}
for i, name in enumerate(names_event):
    temp_dict[name] = names_event_values[i]


event_table_pd = pd.DataFrame(temp_dict)
names_event_values.clear()    
event_table_pd

,run,subrun,event
0,7004,331,16551
1,7004,331,16552
2,7004,331,16553
3,7004,331,16554
4,7004,331,16555
...,...,...,...
1080,7020,1439,72000
1081,5078,78,3904
1082,5078,78,3916
1083,5078,78,3941


In [7]:
len(event_table_pd)

1085

In [86]:
my_index=1
this_run=event_table_pd["run"][my_index]
this_subrun=event_table_pd["subrun"][my_index]
this_event=event_table_pd["event"][my_index]

evt_id = [this_run, this_subrun, this_event]

In [87]:
#montar a tabela de wire

max=20000 #tentar ver depois como pegar evento por evento melhor!!!!

wire_table=f[labels[10]]
event_wire_plan=wire_table["event_id"][0:max]
local_plane = wire_table["local_plane"][0:max]
adc=wire_table["adc"][0:max]
adc = [adc.tolist() for adc in adc]

names_event=["run","subrun","event"]
names_event_values=[[] for _ in range(len(names_event))]
for i in range(max):
    for j in range(len(names_event)):
        names_event_values[j].append(wire_table["event_id"][i][j])

temp_dict = {}
for i, name in enumerate(names_event):
    temp_dict[name] = names_event_values[i]

temp_dict["local_plane"]=local_plane.flatten()
temp_dict["adc"]=adc


wire_table_pd = pd.DataFrame(temp_dict)

adc.clear()
event_wire_plan=None
local_plane=None
wire_table=None

In [88]:
#vamos filtar por eventos, depois esse bloco tem que ser um for, junto com a parte de cima ... uhmmm
wire_table_pd=wire_table_pd.query("run==@this_run and subrun==@this_subrun and event==@this_event")

In [89]:
planeadcs=[]
for p in range(0,nplanes()):
    table_aux=wire_table_pd.query("local_plane==@p")
    aux=[]
    for j in range(len(table_aux)):
        aux.append(table_aux.iloc(0)[j]["adc"])
    planeadcs.append(np.array(aux))

wire_table_pd=wire_table_pd.drop(columns="adc")

In [90]:
from skimage.measure import block_reduce
f_downsample = 6
for p in range(0,nplanes()):
    planeadcs[p] = block_reduce(planeadcs[p], block_size=(1,f_downsample), func=np.sum)

adccutoff = 10.*f_downsample/6.
adcsaturation = 100.*f_downsample/6.
for p in range(0,nplanes()):
    planeadcs[p][planeadcs[p]<adccutoff] = 0
    planeadcs[p][planeadcs[p]>adcsaturation] = adcsaturation

In [91]:
zmax = adcsaturation

print("Run / Sub / Event : %i / %i / %i - saturation set to ADC sum=%.2f" % (evt_id[0], evt_id[1], evt_id[2], zmax))

# Plota a primeira imagem
plt.imshow(planeadcs[0].T, vmin=0, vmax=zmax, origin='lower', cmap='jet')
# Salva a primeira imagem
plt.tight_layout()
plt.savefig(f"./figuras/event_{evt_id[0]}_{evt_id[1]}_{evt_id[2]}_plane_0.png", bbox_inches='tight')
plt.clf()  # Limpa a figura para a próxima imagem

# Plota a segunda imagem
plt.imshow(planeadcs[1].T, vmin=0, vmax=zmax, origin='lower', cmap='jet')
# Salva a segunda imagem
plt.tight_layout()
plt.savefig(f"./figuras/event_{evt_id[0]}_{evt_id[1]}_{evt_id[2]}_plane_1.png", bbox_inches='tight')
plt.clf()  # Limpa a figura para a próxima imagem

# Plota a terceira imagem
plt.imshow(planeadcs[2].T, vmin=0, vmax=zmax, origin='lower', cmap='jet')
# Salva a terceira imagem
plt.tight_layout()
plt.savefig(f"./figuras/event_{evt_id[0]}_{evt_id[1]}_{evt_id[2]}_plane_2.png", bbox_inches='tight')
plt.clf()  # Limpa a figura para a próxima imagem


Run / Sub / Event : 7004 / 331 / 16552 - saturation set to ADC sum=100.00


<Figure size 640x480 with 0 Axes>

In [106]:
#vamos mexer na hit table id
hit_table=f[labels[2]]
event_hit_table=hit_table["event_id"]
names_event=["run","subrun","event"]

#buscar o inicio do evento e o final.
init_hit=0 #lembra depois no for de mudar os paramentros iniciais, para ser mais rapido
end_hit=0 #lembra depois no for de mudar os paramentros iniciais, para ser mais rapido

found_flag=False

#procurando os indices
for aux_hit,values in enumerate(event_hit_table):
    if values[0]==this_run and values[1]==this_subrun and values[2]==this_event and found_flag==False:
        init_hit=aux_hit
        found_flag=True
    if (values[0]!=this_run or values[1]!=this_subrun or values[2]!=this_event) and found_flag==True:
        end_hit=aux_hit
        found_flag=False
        break
#----
names_event_values=[[] for _ in range(len(names_event))]
for i in range(end_hit-init_hit):
    for j in range(len(names_event)):
        names_event_values[j].append(event_hit_table[i+init_hit][j])

temp_dict = {}
for i, name in enumerate(names_event):
    temp_dict[name] = names_event_values[i]

hit_id=hit_table["hit_id"][init_hit:end_hit]
local_plane=hit_table["local_plane"][init_hit:end_hit]
local_time=hit_table["local_time"][init_hit:end_hit]
local_wire=hit_table["local_wire"][init_hit:end_hit]
rms=hit_table["rms"][init_hit:end_hit]

temp_dict["hit_id"]=hit_id.flatten()
hit_id=None
temp_dict["local_plane"]=local_plane.flatten()
local_plane=None
temp_dict["local_time"]=local_time.flatten()
local_time=None
temp_dict["local_wire"]=local_wire.flatten()
local_wire=None
temp_dict["rms"]=rms.flatten()
rms=None

hit_table=None
event_hit_table=None

hit_table_pd = pd.DataFrame(temp_dict)

In [107]:
#vamos mexer na edep table id
edep_table=f[labels[0]]
event_edep_table=edep_table["event_id"]
names_event=["run","subrun","event"]

#buscar o inicio do evento e o final.
init_edep=140 #lembra depois no for de mudar os paramentros iniciais, para ser mais rapido
end_edep=0 #lembra depois no for de mudar os paramentros iniciais, para ser mais rapido

found_flag=False

#procurando os indices # colocar logica de nao achar o proximo
for aux_edep,values in enumerate(event_edep_table[init_edep::]):
    aux_edep=aux_edep+init_edep
    if values[0]==this_run and values[1]==this_subrun and values[2]==this_event and found_flag==False:
        init_edep=aux_edep
        found_flag=True
    if (values[0]!=this_run or values[1]!=this_subrun or values[2]!=this_event) and found_flag==False: #esse aqui eh caso o proximo nao exista
        end_edep=aux_edep
        found_flag=False
        break
    if (values[0]!=this_run or values[1]!=this_subrun or values[2]!=this_event) and found_flag==True:
        end_edep=aux_edep
        found_flag=False
        break

#----
names_event_values=[[] for _ in range(len(names_event))]
for i in range(end_edep-init_edep):
    for j in range(len(names_event)):
        names_event_values[j].append(event_edep_table[i+init_edep][j])

temp_dict = {}
for i, name in enumerate(names_event):
    temp_dict[name] = names_event_values[i]

energy_fraction=edep_table["energy_fraction"][init_edep:end_edep]
hit_id=edep_table["hit_id"][init_edep:end_edep]
g4_id=edep_table["g4_id"][init_edep:end_edep]

temp_dict["energy_fraction"]=energy_fraction.flatten()
energy_fraction=None
temp_dict["hit_id"]=hit_id.flatten()
hit_id=None
temp_dict["g4_id"]=g4_id.flatten()
g4_id=None

edep_table=None
event_edep_table=None

edep_table_pd = pd.DataFrame(temp_dict)


In [108]:
edep_table_pd

,run,subrun,event,energy_fraction,hit_id,g4_id
0,7004,331,16552,1.0,1115,378
1,7004,331,16552,1.0,1119,378
2,7004,331,16552,1.0,1124,378
3,7004,331,16552,1.0,1422,378
4,7004,331,16552,1.0,1822,862
...,...,...,...,...,...,...
2039,7004,331,16552,1.0,17680,354
2040,7004,331,16552,1.0,17683,354
2041,7004,331,16552,1.0,17687,354
2042,7004,331,16552,1.0,17690,354


In [109]:
edep_table_pd = edep_table_pd.sort_values(by=['energy_fraction'], ascending=False, kind='mergesort').drop_duplicates(["hit_id"])
hit_table_pd = hit_table_pd.merge(edep_table_pd, on=["hit_id"], how="left")
hit_table_pd['g4_id'] = hit_table_pd['g4_id'].fillna(-1)
hit_table_pd = hit_table_pd.fillna(0)
hit_table_pd

,run_x,subrun_x,event_x,hit_id,local_plane,local_time,local_wire,rms,run_y,subrun_y,event_y,energy_fraction,g4_id
0,7004,331,16552,0,0,903.091797,0,2.488400,0.0,0.0,0.0,0.0,-1.0
1,7004,331,16552,1,0,910.019897,0,4.152784,0.0,0.0,0.0,0.0,-1.0
2,7004,331,16552,2,0,2006.532471,0,3.619054,0.0,0.0,0.0,0.0,-1.0
3,7004,331,16552,3,0,3721.667969,0,5.285796,0.0,0.0,0.0,0.0,-1.0
4,7004,331,16552,4,0,5921.445312,0,2.916591,0.0,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19089,7004,331,16552,19089,2,5214.856445,3450,2.845015,0.0,0.0,0.0,0.0,-1.0
19090,7004,331,16552,19090,2,5215.486816,3451,2.816978,0.0,0.0,0.0,0.0,-1.0
19091,7004,331,16552,19091,2,5215.805176,3452,3.027444,0.0,0.0,0.0,0.0,-1.0
19092,7004,331,16552,19092,2,5217.264648,3453,2.920720,0.0,0.0,0.0,0.0,-1.0


In [118]:
from math import floor, ceil
planetruth = [np.zeros(shape=(nwires(p),ntimeticks())) for p in range(0,nplanes())]

nrms = 2
for p in range(0,nplanes()):
    nuhits = hit_table_pd.query('local_plane==%i and g4_id>=0'%p)[['local_wire','local_time','rms']]
    for i,h in nuhits.iterrows():
        print(floor(h['local_time']-nrms*h['rms']),ceil(h['local_time']+nrms*h['rms']))
        planetruth[p][int(h['local_wire'])][floor(h['local_time']-nrms*h['rms']):ceil(h['local_time']+nrms*h['rms'])] = 1
    cosmhits = hit_table_pd.query('local_plane==%i and g4_id<0'%p)[['local_wire','local_time','rms']]
    for i,h in cosmhits.iterrows():
        planetruth[p][int(h['local_wire'])][floor(h['local_time']-nrms*h['rms']):ceil(h['local_time']+nrms*h['rms'])] = -1

for p in range(0,nplanes()):
    planetruth[p] = block_reduce(planetruth[p], block_size=(1,f_downsample), func=np.sum)

""" for p in range(0,nplanes()):
    planetruth[p] = np.multiply(planetruth[p],planeadcs[p]) """

1928 1943
1927 1943
1926 1942
1873 1890
3487 3503
3485 3501
3483 3497
2434 2455
4096 4112
4095 4113
3387 3400
3385 3400
4326 4346
4328 4343
4091 4125
4097 4125
4113 4132
4300 4317
2569 2585
4302 4318
2570 2588
2574 2593
2579 2598
2584 2603
3101 3116
2589 2611
2596 2619
2604 2626
3347 3360
2611 2632
3346 3360
2618 2638
3345 3358
2624 2645
2630 2651
2636 2657
2643 2663
2649 2670
2656 2678
2663 2686
2671 2696
2680 2699
2684 2701
4055 4075
4063 4079
4144 4160
4143 4160
4142 4160
3674 3698
3679 3721
3689 3758
4012 4030
3675 3695
3682 3709
3684 3739
3687 3801
4008 4039
3668 3690
3684 3706
3700 3722
3716 3738
3732 3754
3748 3770
3764 3786
3780 3802
3796 3818
3812 3834
3828 3850
3844 3866
3860 3882
3876 3898
3892 3914
3908 3930
3924 3946
3942 3950
4010 4023
4014 4039
4028 4049
3807 3821
3828 3849
3849 3862
3894 3907
3910 3923
3926 3944
3803 3832
3811 3864
3852 3865
3925 3945
3924 3941
3952 3990
3920 3939
3969 4005
3916 3936
3977 4000
3993 4007
3986 4032
3913 3931
3997 4024
4008 4044
3909 3927


' for p in range(0,nplanes()):\n    planetruth[p] = np.multiply(planetruth[p],planeadcs[p]) '

In [119]:
ntimeticks()

6400

In [111]:
cosmhits

,local_wire,local_time,rms
14595,2,1859.483643,2.663851
14596,3,1862.487061,3.425622
14597,4,1860.802002,3.611362
14598,5,1872.895020,4.504058
14599,6,1885.371460,4.241106
...,...,...,...
19089,3450,5214.856445,2.845015
19090,3451,5215.486816,2.816978
19091,3452,5215.805176,3.027444
19092,3453,5217.264648,2.920720


In [112]:
nuhits

,local_wire,local_time,rms
16847,1446,3690.280518,7.093081
16848,1447,3680.921631,6.909979
16948,1546,2922.108398,4.541143
16949,1547,2916.538086,4.871610
16950,1548,2911.551514,4.927672
...,...,...,...
17680,2054,2584.257080,3.643610
17683,2055,2582.055176,3.616456
17687,2056,2579.802490,3.634216
17690,2057,2577.681885,3.648571


In [117]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20, 15), dpi=600)

im1 = ax1.imshow(planetruth[0].T,vmin=-1,vmax=1,origin='lower',cmap='coolwarm')
ax1.set_title("Plane 0")
ax1.set_xlabel("Wire")
ax1.set_ylabel("Time Tick")

im2 = ax2.imshow(planetruth[1].T,vmin=-1,vmax=1,origin='lower',cmap='coolwarm')
ax2.set_title("Plane 1")
ax2.set_xlabel("Wire")

im3 = ax3.imshow(planetruth[2].T,vmin=-1,vmax=1,origin='lower',cmap='coolwarm')
ax3.set_title("Plane 2")
ax3.set_xlabel("Wire")

plt.tight_layout()

In [100]:
zmax = adcsaturation

print("Run / Sub / Event : %i / %i / %i - saturation set to ADC sum=%.2f"%(evt_id[0],evt_id[1],evt_id[2],zmax))

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20, 15), dpi=600)

im1 = ax1.imshow(planeadcs[0].T,vmin=0,vmax=zmax,origin='lower',cmap='jet')
ax1.set_title("Plane 0")
ax1.set_xlabel("Wire")
ax1.set_ylabel("Time Tick")

im2 = ax2.imshow(planeadcs[1].T,vmin=0,vmax=zmax,origin='lower',cmap='jet')
ax2.set_title("Plane 1")
ax2.set_xlabel("Wire")

im3 = ax3.imshow(planeadcs[2].T,vmin=0,vmax=zmax,origin='lower',cmap='jet')
ax3.set_title("Plane 2")
ax3.set_xlabel("Wire")

plt.tight_layout()

Run / Sub / Event : 7004 / 331 / 16552 - saturation set to ADC sum=100.00
